In [57]:
import pandas as pd
from database import dataRefine
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from binascii import Incomplete
from collections import defaultdict
from typing import IO
import re

In [86]:
slip = pd.read_excel('../sample_data/KJSIT_MAY_2023.xlsx')
dataRefine(slip)
print(slip[slip['hr_emp_code']==220309])

   hr_emp_code       name_of_staff  org_basic  basic_salary  agp_or_gp  \
0       220309  Dr Vivek Sunnapwar     100000        120000       5000   

   da_0_wef_april_2023  hra_24   cla    ta  vehicle_and_driver_all  ...  lic  \
0                    0   24000  2000  3000                    5000  ...  800   

   personal_loan  staff_loan_or_diwali_festival  staff_advance  \
0           5000                           1000           2000   

   medical_policy_april_23_to_mar_24  kjsieit_empoyees_coop_credit_socity_ltd  \
0                               4000                                     2000   

   union_fees  other_deduction_for_salary_paid  total_deduction  net_salary  
0         100                             1500            15000      165500  

[1 rows x 31 columns]


In [114]:
def mapping(pd_columns:pd.DataFrame,columns:str) -> dict[str,str]:

    check_columns = columns.lower().split(' ')

    memo=defaultdict(lambda: {'count':0,'col':None})
    to_check = [col for col in pd_columns if any(word in col for word in check_columns)]

    for col in to_check:
        for check in check_columns:
            asq = 0
            
            if check in col:
                asq += 1

        if memo[columns]['count']<asq:
            memo[columns]['col']=col
            memo[columns]['count']=asq

    return {columns:memo[columns]['col']}

def check_column(col:str,pd_columns:list[str],pd_Data:pd.DataFrame) -> str:
    memo = mapping(pd_columns,col)

    if col in memo and memo[col]:
        return pd_Data[memo[col]].values[0]
    else: 
        return 'None'
    
check_column(pd_Data=slip[slip['hr_emp_code']==220309],pd_columns=slip.columns,col='emp code')

"""

import pandas as pd
from fuzzywuzzy import fuzz

def check_column_efficient(col: str, pd_columns: list[str], pd_data: pd.DataFrame) -> str:

    col_lower = col.lower() 

    filtered_cols = [c for c in pd_columns if any(w in c.lower() for w in col_lower.split())]

    if filtered_cols:

        distances = {c: fuzz.ratio(col_lower, c.lower()) for c in filtered_cols}

        best_match = max(distances, key=distances.get)

        return pd_data[best_match].values[0]

    else:
        return 'None'
        
"""

{'emp code': 'hr_emp_code'}


"\n\nimport pandas as pd\nfrom fuzzywuzzy import fuzz\n\ndef check_column_efficient(col: str, pd_columns: list[str], pd_data: pd.DataFrame) -> str:\n\n    col_lower = col.lower() \n\n    filtered_cols = [c for c in pd_columns if any(w in c.lower() for w in col_lower.split())]\n\n    if filtered_cols:\n\n        distances = {c: fuzz.ratio(col_lower, c.lower()) for c in filtered_cols}\n\n        best_match = max(distances, key=distances.get)\n\n        return pd_data[best_match].values[0]\n\n    else:\n        return 'None'\n        \n"

In [115]:
#svv

def generate_pdf(self,employee_data:pd.DataFrame,month:str,year:int,type:str,chosen:str,file:IO[Incomplete],bulk:bool) -> None:  

    pd_columns = sorted(employee_data.columns,key=len)

    emp_data = {
        "name":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="name"),
        "designation":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="designation"),
        "pf_no":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="pf no"),
        "hra":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="hra rent"),
        "pan":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="pan no"),
        "date":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="date of joining"),
        "days":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="no of days present"),
        "emp_code":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="hr emp code"),
        "salary_ac_no":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="salary a / c no"),
        "aadhar":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="aadhar card no"),
        "una":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="una"),
        "basic":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="basic pay salary"),
        "present_basic":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="present basic pay salary"),
        "ta":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="ta"),
        "prof_tax":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="prof tax"),
        "pf":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="pf"),
        "tds":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="tds"),
        "lic":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="lic"),
        "special_alw":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="special allowance alw"),
        "salary_arr":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="salary arrears"),
        "vehicle":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="vehicle all"),
        "books":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="books periodicals client entertainment"),
        "etn":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="entertainment allowance allw"),
        "petrol":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="petrol"),
        "telephone":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="telephone"),
        "medical":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="medical policy"),
        "pba":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="pba allowance alw"),
        "lta":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="lta"),
        "ex-grataia":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="ex grataia"),
        "ent":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="ent alw allowance"),
        "gross":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="gross salary"),
        "net_salary":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="net salary payable"),
        "principle":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="principle loan PM amount"),
        "interest":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="interest bal amount"),
        "other_ded":check_column(pd_columns=pd_columns,pd_Data=employee_data,col="other deduction"),
        'total_ded':check_column(pd_columns=pd_columns,pd_Data=employee_data,col='total deduction')

    }

    pdf_file = f"{chosen}_{type}_{month}_{year}_employee_{emp_data['emp_code']}.pdf"

    
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    elements = []

    styles = getSampleStyleSheet()
    style_normal = styles["Normal"]
            
    # Title
    title_data = [
        ["K.J SOMAIYA INSTITUTE OF TECHNOLOGY, SOMAIYA AYURVIHAR EVARAD NAGAR, EASTERN EXPRESS HIGHWAY SION"],
        [f"PAY SLIP FOR THE MONTH OF {month.capitalize()}-{year}     31 DAYS     1"]
    ]
    title_table = Table(title_data, colWidths=[540])
    title_table.setStyle(TableStyle([
        ("SPAN", (0, 0), (0, 0)),
        ("SPAN", (0, 1), (0, 1)),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, -1), "Helvetica-Bold")
    ]))
    elements.append(title_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing


    # Employee Info
    employee_info_data = [
        ["NAME", emp_data["name"], "DESIGNATION", emp_data["designation"]],
        ["DATE OF JOINING", emp_data["date"], "NO OF DAYS PRESENT", emp_data["days"]],
        ["PF NO", emp_data["pf_no"], "PAN NO", emp_data["pan"]],
        ["EMP CODE", emp_data["emp_code"], "SALARY A/C NO", emp_data["salary_ac_no"]],
        ["Aadhar Card No", emp_data["aadhar"], "UNA", emp_data["una"]],
    ]

    employee_info_table = Table(employee_info_data, colWidths=[120, 160, 120, 160])
    employee_info_table.setStyle(TableStyle([
        ("GRID", (0, 0), (-1, -1), 0.5, colors.black),
        ("ALIGN", (0, 0), (-1, -1), "LEFT"),
        ("FONTNAME", (0, 0), (-1, -1), "Helvetica-Bold")
    ]))
    elements.append(employee_info_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing

    # Earnings and Deductions
    earnings_deductions_data = [
        ["EARNINGS", "Rs.", "DEDUCTIONS", "Rs."],
        ["Basic Pay", emp_data["basic"], "PROF TAX", emp_data["prof_tax"]],
        ["Present Basic Pay", emp_data["present_basic"], "PF", emp_data["pf"]],
        ["RENT HRA 30%", emp_data["hra"],  "TDS", emp_data["tds"]],
        ["TA / Conveyance", emp_data["ta"], "LIC", emp_data["lic"]],
        ["SPECIAL ALW", emp_data["special_alw"], ""],
        ["Salary Arrears", emp_data["salary_arr"], "Principle loan amount PM", emp_data["principle"]],
        ["Vehicle", emp_data["vehicle"], "Interest 6% on bal amount", emp_data["interest"]],

        ["Books and Periodicals and\n Client Entertainment exp",emp_data["books"], "Other Deduction", emp_data["other_ded"]],
        ["Etn Alw", emp_data["etn"], "", ""],
        ["Petrol Alw", emp_data['petrol'], "", ""],
        ["Telephone", emp_data['telephone'], "",""],
        ["Medical", emp_data['medical'], "", ""],
        ["PBA All", emp_data['pba'], "", ""],
        ["LTA Alw", emp_data['lta'], "", ""],
        ["EX-Grataia", emp_data['ex-grataia'], "", ""],
        ["Ent All", emp_data['ent'], "", ""],
        ["GROSS SALARY", emp_data['gross'], "TOTAL DEDUCTION", emp_data['total_ded']],
        ["NET SALARY PAYABLE", emp_data['net_salary'], "", ""]
    ]
    
    earnings_deductions_table = Table(earnings_deductions_data, colWidths=[180, 100, 180, 100])
    earnings_deductions_table.setStyle(TableStyle([
        ("GRID", (0, 0), (-1, -1), 0.5, colors.black),
        ("BACKGROUND", (0, 0), (-1, 0), colors.lightgrey),
        ("ALIGN", (0, 0), (-1, -1), "CENTER")
    ]))
    elements.append(earnings_deductions_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing

    # Footer
    footer_data = [
        ["This is a computer generated salary slip", ""]
    ]
    footer_table = Table(footer_data, colWidths=[300, 240])
    footer_table.setStyle(TableStyle([
        ("SPAN", (0, 0), (-1, 0)),
        ("ALIGN", (0, 0), (0, 0), "CENTER"),
        ("ALIGN", (0, 1), (0, 1), "RIGHT"),
        ("FONTNAME", (0, 0), (-1, -1), "Helvetica-Bold")
    ]))
    elements.append(footer_table)

    doc.build(elements)

generate_pdf(self=None,employee_data=pd.DataFrame(slip[slip['hr_emp_code']==220309]),month="",bulk=None,chosen=None,year=None,type="svv",file=None)

{'name': 'name_of_staff'}
{'designation': None}
{'pf no': None}
{'hra rent': None}
{'pan no': None}
{'date of joining': None}
{'no of days present': None}
{'hr emp code': 'hr_emp_code'}
{'salary a / c no': None}
{'aadhar card no': None}
{'una': None}
{'basic pay salary': 'net_salary'}
{'present basic pay salary': 'net_salary'}
{'ta': 'ta'}
{'prof tax': 'prof_tax'}
{'pf': 'pf_rs780_or_to1800_from_sep_2014'}
{'tds': 'tds'}
{'lic': 'lic'}
{'special allowance alw': 'lta_alw'}
{'salary arrears': None}
{'vehicle all': 'vehicle_and_driver_all'}
{'books periodicals client entertainment': 'entertainment_allwper_mont'}
{'entertainment allowance allw': 'entertainment_allwper_mont'}
{'petrol': None}
{'telephone': 'telephone'}
{'medical policy': 'medical_policy_april_23_to_mar_24'}
{'pba allowance alw': 'lta_alw'}
{'lta': 'lta_alw'}
{'ex grataia': 'exgrataia'}
{'ent alw allowance': 'salary_arrers_or_special_other_allowance_or_rem'}
{'gross salary': 'net_salary'}
{'net salary payable': None}
{'princ

In [ ]:
#kjs

def generate_pdf(self,employee_data:pd.DataFrame,month:str,year:int,type:str,chosen:str,file:IO[Incomplete],bulk:bool) -> None:  

    pdf_file = f"{chosen}_{type}_{month}_{year}_employee_{employee_data[0]}.pdf"

    
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    elements = []

    styles = getSampleStyleSheet()
    style_normal = styles["Normal"]
            
    # Title
    title_data = [
        ["K.J SOMAIYA INSTITUTE OF TECHNOLOGY, SOMAIYA AYURVIHAR EVARAD NAGAR, EASTERN EXPRESS HIGHWAY SION"],
        [f"PAY SLIP FOR THE MONTH OF {month.capitalize()}-{year}     31 DAYS     1"]
    ]
    title_table = Table(title_data, colWidths=[540])
    title_table.setStyle(TableStyle([
        ('SPAN', (0, 0), (0, 0)),
        ('SPAN', (0, 1), (0, 1)),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold')
    ]))
    elements.append(title_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing

    # Employee Info
    employee_info_data = [
        ["NAME", employee_data[1], "DESIGNATION", employee_data[2]],
        ["DATE OF JOINING", employee_data[3], "NO OF DAYS PRESENT", employee_data[4]],
        ["PF NO", employee_data[5], "PAN NO", employee_data[6]],
        ["EMP CODE", employee_data[0], "SALARY A/C NO", employee_data[7]],
        ["Aadhar Card No", employee_data[8], "UNA", employee_data[9]],
    ]
    employee_info_table = Table(employee_info_data, colWidths=[120, 160, 120, 160])
    employee_info_table.setStyle(TableStyle([
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold')
    ]))
    elements.append(employee_info_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing

    # Earnings and Deductions
    earnings_deductions_data = [
        ["EARNINGS", "Rs.", "DEDUCTIONS", "Rs."],
        ["Basic Pay", employee_data[10], "PROF TAX", employee_data[26]],
        ["Basic", employee_data[10], "", ""],
        ["RENT HRA", employee_data[11], "PF", employee_data[27]],
        ["30%", employee_data[11], "TDS", employee_data[28]],
        ["TA/ Conveyance", employee_data[12], "LIC", employee_data[29]],
        ["SPECIAL ALW", employee_data[13], ""],
        ["Salary Arrears", employee_data[14], "Principle loan amount PM", employee_data[30]],
        ["Vehicle", employee_data[15], "", ""],
        ["Books and Periodicals", employee_data[16], "Interest 6% on bal amount", employee_data[30]],
        ["Etn Alw", employee_data[17], "", ""],
        ["Petrol Alw", employee_data[18], "", ""],
        ["Telephone", employee_data[19], "Other Deduction", employee_data[30]],
        ["Medical", employee_data[20], "", ""],
        ["LTA", employee_data[21], "", ""],
        ["Alw", employee_data[22], "", ""],
        ["EX-Grataia", employee_data[23], "", ""],
        ["Ent All", employee_data[24], "", ""],
        ["GROSS SALARY", employee_data[25], "TOTAL DEDUCTION", employee_data[30]],
        ["NET SALARY PAYABLE", employee_data[30], "", ""]
    ]
    earnings_deductions_table = Table(earnings_deductions_data, colWidths=[180, 100, 180, 100])
    earnings_deductions_table.setStyle(TableStyle([
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER')
    ]))
    elements.append(earnings_deductions_table)
    elements.append(Paragraph("<br/><br/>", style_normal))  # Add spacing

    # Footer
    footer_data = [
        ["This is a computer generated salary slip", ""]
    ]
    footer_table = Table(footer_data, colWidths=[300, 240])
    footer_table.setStyle(TableStyle([
        ('SPAN', (0, 0), (-1, 0)),
        ('ALIGN', (0, 0), (0, 0), 'CENTER'),
        ('ALIGN', (0, 1), (0, 1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold')
    ]))
    elements.append(footer_table)

    doc.build(elements)
